## Лабораторная работа 2
### Выборочное наблюдение
1. Из генеральной совокупности сгенерировать 36 выборок одинакового
объема, для каждой из которых найти выборочную среднюю. Объем
определить по схеме повторной выборки так, чтобы с надежностью γ = 0,95
обеспечить точность оценки математического ожидания δ = 3 года.
2. На основании полученных значений выборочных средних построить
интервальный ряд распределения, в котором левая граница – округленное
вниз минимальное значение выборочной средней, правая граница –
округленное вверх максимальное значение выборочной средней, длина
интервала – 1 год. Вычислить относительные частоты, построить
гистограмму.


In [ ]:
import matplotlib.pyplot as plt
from math import *
import numpy as np



### Статистические оценки параметров распределения
1. Полагая, что распределение среднего возраста подчиняется
нормальному закону, осуществить выравнивание статистического ряда.
Найти точечные оценки параметров методом моментов. Построить кривую
Гаусса, аппроксимировав ею гистограмму частот.
2. На основании одной из сгенерированных выборок найти
доверительный интервал для оценки математического ожидания случайной
величины «возраст» с надежностью 0,95, предполагая, что среднее
квадратическое отклонение неизвестно